## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [95]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    
                    
                if name in list(check_sheet.Workflow):
                    print(name)
                    dfy = check_sheet[check_sheet['Workflow']== name]
            
                    dfy.reset_index(drop=True)

                    for index,curie_id in enumerate(dfy.Curie):
                        print(index,curie_id)
                        node_num = dfy.iloc[index][3]
                        query_id = curie_id
                        if np.isnan(dfy.iloc[index][2]) == True:
                            len_check = len(child_response['fields']['data']['message']['results'])
                        else:
                            len_check = dfy.iloc[index][2]

                        #print(node_num, query_id, len_check)

                        locs = []
                        for x, val in enumerate(child_response['fields']['data']['message']['results']):
                            #print(val)

                            if x < len_check:

                                if query_id in val['node_bindings'][node_num][0]['id']:
                                    locs.append(x)
                        if not locs:
                            check_result = f'False'
                            print(check_result)
                            #pass
                        else:
                            check_result = f'True'
                            print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                        dict3[curie_id] = check_result    
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [97]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [5]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(400)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=4bbb90d4-5ad1-44f8-af3d-28786a77dfce
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 168
Done
ara-bte Done 133
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=1648dfdd-4435-480f-a19f-e9bbc202c989
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 0
Done
ara-bte Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=1a84bd2f-7b66-4f2b-86e2-0a8a48c01e23
Done
Done
ara-arax Done 400
Done
ara-aragorn ARS Error 0
Done
ara-bte Done 83
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.Multiomics_BigGIM_DR_KP_RHOBTB2.json
A.Multiomics_BigGIM_DR_KP_RHOBTB2
https://arax.ncats.io/?source=ARS&id=8e337f4c-77a2-4a89-9a7b-d59a133bdf35
Done
Done


https://arax.ncats.io/?source=ARS&id=6e9e91bc-b5b2-4cc9-b17c-bcc6dd106bb1
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 100
Done
ara-bte Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
https://arax.ncats.io/?source=ARS&id=35b6de80-e92a-49c0-8d9e-d9bb60e57e56
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 140
Done
ara-bte Done 114
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.3a_MultSclerosis_related_to_Nimodipine.json
C.3a_MultSclerosis_related_to_Nimodipine
https://arax.ncats.io/?source=ARS&id=d41fd067-e0dc-4cfb-a414-79d631ed2bf6
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 1
Done
ara-bte Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
https://arax.ncats.io/?sou

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [6]:
dict_workflows

{'A.3_KCNMA1': '4bbb90d4-5ad1-44f8-af3d-28786a77dfce',
 'A.2a_RHOBTB2_twohop': '1648dfdd-4435-480f-a19f-e9bbc202c989',
 'A.8_EGFR_simple': '1a84bd2f-7b66-4f2b-86e2-0a8a48c01e23',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '8e337f4c-77a2-4a89-9a7b-d59a133bdf35',
 'A.0_RHOBTB2_direct': '32a6e68f-4e83-4a63-a738-1a02c7854bc9',
 'A.2_RHOBTB2_twohop': 'efd74cb0-2f35-4f02-a4b9-b4e53bb6fefa',
 'A.Multiomics_BigGIM_DR_KP_EGFR': 'c40713c2-b197-4e6a-9252-583a81dfe4c2',
 'A.2_RHOBTB2_twohop_constrained': '60dda5d2-e791-447f-9921-a837f76f7086',
 'A.9_EGFR_advanced': '7ebdbefa-0624-49ab-a007-c7f8447b962d',
 'A.2a_expanded_RHOBTB2_twohop_constrained': '05e6c956-fb74-43c8-8014-dd4833fc983c',
 'A.1_RHOBTB2': '4bb91964-83d5-47b4-8d49-00c5b4f2ceb3',
 'A.2a_expanded_RHOBTB2_twohop': '4e102dc4-9cde-4ce4-90a8-e4ab92de57a2',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': 'b3a2e6ce-f05b-4530-8460-952600a907aa',
 'B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': '16474c90-38d0-44be-9284-87

In [98]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1.json 4bbb90d4-5ad1-44f8-af3d-28786a77dfce
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 168
Done
ara-bte Done 133
A.2a_RHOBTB2_twohop.json 1648dfdd-4435-480f-a19f-e9bbc202c989
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 0
Done
ara-bte Done 0
A.8_EGFR_simple.json 1a84bd2f-7b66-4f2b-86e2-0a8a48c01e23
Done
Done
ara-arax Done 400
Done
ara-aragorn ARS Error 0
Done
ara-bte Done 83
A.Multiomics_BigGIM_DR_KP_RHOBTB2.json 8e337f4c-77a2-4a89-9a7b-d59a133bdf35
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 4
Done
ara-bte Done 4
A.0_RHOBTB2_direct.json 32a6e68f-4e83-4a63-a738-1a02c7854bc9
Done
Done
ara-aragorn ARS Error 0
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
ara-arax ARS Error 0
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
ara-bte ARS Error 0
A.2_RHOBTB2_twohop.json efd74cb0-2f35-4f02-a4b9-b4e53bb6fefa
Done
Done
ara-arax Done 500
Done
ara-aragorn ARS Error 0
Done
ara-bte Done 1499
A.Multiomics_BigGIM_DR_KP_EGFR.json c40713c2-b197-4e6a-

### Creating dataframe for workflows with PK

<br>

In [99]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-interacts-with
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3_DILI_branched-four-hop
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_t

In [100]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [101]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=32a6e68f-...,https://arax.ncats.io/?source=ARS&id=4bb91964-...,https://arax.ncats.io/?source=ARS&id=efd74cb0-...,https://arax.ncats.io/?source=ARS&id=60dda5d2-...,https://arax.ncats.io/?source=ARS&id=1648dfdd-...,https://arax.ncats.io/?source=ARS&id=4e102dc4-...,https://arax.ncats.io/?source=ARS&id=05e6c956-...,https://arax.ncats.io/?source=ARS&id=4bbb90d4-...,https://arax.ncats.io/?source=ARS&id=1a84bd2f-...,https://arax.ncats.io/?source=ARS&id=7ebdbefa-...,...,https://arax.ncats.io/?source=ARS&id=35b6de80-...,https://arax.ncats.io/?source=ARS&id=cd161bbc-...,https://arax.ncats.io/?source=ARS&id=6e9e91bc-...,https://arax.ncats.io/?source=ARS&id=119f0d49-...,https://arax.ncats.io/?source=ARS&id=d41fd067-...,https://arax.ncats.io/?source=ARS&id=e765fe4a-...,https://arax.ncats.io/?source=ARS&id=c34b03ff-...,https://arax.ncats.io/?source=ARS&id=884f8e91-...,https://arax.ncats.io/?source=ARS&id=b2dfa4f3-...,https://arax.ncats.io/?source=ARS&id=2c87a425-...
ara-aragorn,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-arax,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,No Results: 200,No Results: 200,Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},No Results: 200,No Results: 200,Results: 200{},No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


In [102]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [12]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-interacts-with
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3_DILI_branched-four-hop
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_t

In [13]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-arax': {'infores:arax',
               'infores:ctd',
               'infores:molepro'}},
             'A.1_RHOBTB2': {'ara-arax': {'infores:arax', 'infores:rtx-kg2'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'}},
             'A.2_RHOBTB2_twohop': {'ara-arax': {'infores:arax',
               'infores:connections-hypothesis',
               'infores:ctd',
               'infores:drugbank',
               'infores:molepro',
               'infores:rtx-kg2'},
              'ara-bte': {'infores:automat-covidkop',
               'infores:automat-ctd',
               'infores:automat-gtopdb',
               'infores:automat-hetio',
               'infores:automat-pharos',
               'infores:automat-robokop',
               'infores:biothings-dgidb',
               'infores:biothings-multiomics-biggim-drugresponse',
      

In [14]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [15]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-arax', 'infores:ctd'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:arax'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:ctd'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:connections-hypothesis'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:rtx-kg2'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:drugbank'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:molepro'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:cord19-scibite'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:automat-covidkop'],
 ['A.2_RHOBTB2_twoh

In [16]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [17]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [18]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [19]:
df2test = pd.DataFrame(df2test.unstack().T)

In [20]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-arax],[ara-bte],NaN,[ara-bte],[ara-bte],NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[ara-bte],NaN,[ara-bte],[ara-bte],NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:stitch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,"[ara-arax, ara-bte]"
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte]
infores:tcrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],[ara-arax],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax]


In [21]:
#df2test.drop([''], axis=0, inplace=True)

In [22]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [23]:
#df2.replace([], 'None', regex=True,inplace=True)

In [24]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [25]:
df2test = df2test.rename_axis(None)

In [26]:
df2test.columns.name = None

In [27]:
df2test.fillna('', inplace=True)

In [28]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.6_metformin-ferritin
CTD,,,,,,,,,,,...,,,,[ara-arax],[ara-bte],,[ara-bte],[ara-bte],,
DisGeNET,,,,,,,,,,,...,,,,,,,[ara-bte],[ara-bte],,
DrugCentral Indication,,,,,,,,,,,...,,,,,[ara-bte],,[ara-bte],[ara-bte],,
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,,,[ara-bte],,,
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,,[ara-bte],,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:stitch,,,,,,,,,,,...,,,,,,,,[ara-arax],,"[ara-arax, ara-bte]"
infores:string,,,,,,,,,,,...,,,,,,,,,,[ara-bte]
infores:tcrd,,,,,,,,[ara-arax],[ara-arax],,...,,,,,,,,,,[ara-arax]
infores:translator-biothings-explorer,,[ara-bte],[ara-bte],"[ara-arax, ara-bte]",[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],...,[ara-bte],"[ara-arax, ara-bte]",[ara-arax],[ara-arax],"[ara-arax, ara-bte]",,"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]"


In [29]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [30]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:335]
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

        
        
        
        
        
        
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]
df2test['Query Type'] = 'Async'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

In [31]:
df2test

,Query Type,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.6_metformin-ferritin
infores:biothings-diseases,Async,KP,,,,,,,,,...,,,,,,,[ara-bte],,,
infores:molepro,Async,KP,[ara-arax],,[ara-arax],[ara-arax],,,"[ara-arax, ara-bte]",[ara-arax],...,[ara-bte],,,,,,"[ara-arax, ara-bte]","[ara-arax, ara-bte]",[ara-bte],"[ara-arax, ara-bte]"
infores:mychem-info,Async,KP,,,,[ara-arax],,,,,...,,,[ara-arax],,[ara-arax],,,[ara-arax],,[ara-arax]
infores:mydisease-info,Async,KP,,,,,,,,,...,[ara-bte],,,[ara-arax],"[ara-arax, ara-bte]",,[ara-bte],[ara-bte],[ara-bte],
infores:openpredict,Async,KP,,,,,,,,,...,[ara-bte],,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,Async,Illegal value,,,,,,,,,...,[ara-bte],,,,,,[ara-bte],,[ara-bte],
infores:hetio,Async,Illegal value,,,[ara-bte],[ara-bte],,,[ara-bte],,...,[ara-bte],,[ara-arax],[ara-arax],[ara-bte],,[ara-bte],"[ara-arax, ara-bte]",[ara-bte],[ara-arax]
infores:ontological-hierarchy,Async,Illegal value,,,,,,,,,...,,,,,,,[ara-bte],,,[ara-bte]
infores:pdsp,Async,Illegal value,,,,,,,,,...,,,,,,,,[ara-arax],,


<br>

## Converting the Pk's to hyperlink

<br>

In [103]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [104]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [105]:
df.sort_index(inplace=True)

In [106]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [107]:
df

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-arax,Async,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,Async,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
pk,Async,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...",...,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."


In [37]:
#date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
#wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [38]:
#wks_name

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [83]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [84]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [85]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_09_29-08_27_23_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

existing.loc['pk'][:] = existing.loc['pk'][:].apply(lambda x: x.replace('" "', '","'))

existing.loc['pk'][1]

updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True)

In [86]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [87]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_09_29-08_27_23_AM")

In [88]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Unknown'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_CONTAINS',
       'values': [{'userEnteredValue': 'Error: 501'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'text

In [89]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Unknown'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor'

In [77]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [90]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_CONTAINS', ['Error: 501']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [91]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_CONTAINS',
       'values': [{'userEnteredValue': 'Error: 501'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1862962344,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bol

In [92]:
# Select a range
cell_list = sh.range('B19:AS19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_29-08_27_23_AM'!B19:AS19",
 'updatedRows': 1,
 'updatedColumns': 44,
 'updatedCells': 44}

In [93]:
# Select a range
cell_list = sh.range('B23:AS23')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_29-08_27_23_AM'!B23:AS23",
 'updatedRows': 1,
 'updatedColumns': 44,
 'updatedCells': 44}

In [94]:
set_frozen(sh, cols=1, rows=1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

### Push Dataframe 2

<br>

In [50]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_09_29-08_27_23_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws2.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

updated = existing.append(df2test)
gd.set_with_dataframe(ws2, updated,include_index=True,include_column_header=True)


In [51]:
set_frozen(ws2, cols=1, rows =1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}